# Loading data

In [1]:
import json

In [2]:
with open('clinais.train.json') as f:
    data = json.load(f)

In [3]:
from tqdm import tqdm

In [4]:
finalresult = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresult.append([ident,res,tags])

# finalresult    

100%|██████████| 781/781 [00:00<00:00, 6800.72it/s]


In [5]:
import numpy as np
import itertools
tags = [x[2] for x in finalresult]
tags = np.unique(list(itertools.chain(*tags)))
id2label = {}
label2id = {}
for i,tag in enumerate(tags):
    id2label[i] = tag
    label2id[tag] = i

In [6]:
finalresult = [[x[0],x[1],[label2id[y] for y in x[2]]] for x in finalresult]
#finalresult[0]


In [7]:
with open('clinais.dev.json') as f:
    data = json.load(f)

In [8]:
finalresultdev = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresultdev.append([ident,res,tags])

100%|██████████| 127/127 [00:00<00:00, 8176.28it/s]


In [9]:
finalresultdev = [[x[0],x[1],[label2id[y] for y in x[2]]] for x in finalresultdev]


In [10]:
from datasets import Dataset,DatasetDict

In [11]:
import pandas as pd
df = pd.DataFrame(data=finalresult,columns=['id','tokens','tags'])
dataset_train = Dataset.from_pandas(df)

In [12]:
df = pd.DataFrame(data=finalresultdev,columns=['id','tokens','tags'])
dataset_val = Dataset.from_pandas(df)

In [13]:
dataset = DatasetDict(train=dataset_train,val=dataset_val)

# Processing dataset

In [14]:
from transformers import AutoTokenizer

In [15]:
modelCheckpoint = "dccuchile/distilbert-base-spanish-uncased"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 781
    })
    val: Dataset({
        features: ['id', 'tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127
    })
})

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
import evaluate

seqeval = evaluate.load("seqeval")

In [21]:
import numpy as np



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [22]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    modelCheckpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at dccuchile/distilbert-base-spanish-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at dccuchile/distilbert-base-spanish-uncased and are newly initialized: ['classifier.bias', 'classifier.

In [23]:
training_args = TrainingArguments(
    output_dir="clinico",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=300,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/joheras/clinico into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/255M [00:00<?, ?B/s]

Download file runs/Mar11_17-37-13_minion/events.out.tfevents.1678552637.minion.3340830.2:  44%|####3     | 8.1…

Download file runs/Mar11_17-37-13_minion/1678552637.3475356/events.out.tfevents.1678552637.minion.3340830.3: 1…

Download file runs/Mar11_17-54-22_minion/1678553667.5775871/events.out.tfevents.1678553667.minion.3347124.1: 1…

...

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, id, tags. If tokens, id, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/joheras/.local/lib/python3.10/site-packages/transformers/optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 781
  Num Epochs = 300
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 2100
  Number of trainable parameters = 66349070
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
F

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.701133,0.000000,0.000000,0.000000,0.484327
2,No log,1.503659,0.029412,0.023333,0.026022,0.506623
3,No log,1.382401,0.002683,0.003333,0.002973,0.550716
4,No log,1.301496,0.001163,0.001111,0.001136,0.554974
5,No log,1.223113,0.003647,0.005556,0.004403,0.564808
6,No log,1.164821,0.005196,0.010000,0.006839,0.589954
7,No log,1.116485,0.008898,0.020000,0.012316,0.605330
8,No log,1.046988,0.012109,0.032222,0.017602,0.639057
9,No log,0.990870,0.019388,0.050000,0.027942,0.664516
10,No log,0.953524,0.026606,0.058889,0.036653,0.675790


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, id, tags. If tokens, id, tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 128
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to clinico/checkpoint-7
Configuration saved in clinico/checkpoint-7/config.json
Model weights saved in clinico/checkpoint-7/pytorch_model.bin
tokenizer config file saved in clinico/checkpoint-7/tokenizer_config.json
Special tokens file saved in clinico/checkpoint-7/special_tokens_map.json
tok

TrainOutput(global_step=2100, training_loss=0.14952304227011545, metrics={'train_runtime': 2320.27, 'train_samples_per_second': 100.98, 'train_steps_per_second': 0.905, 'total_flos': 3.06186633713664e+16, 'train_loss': 0.14952304227011545, 'epoch': 300.0})

In [24]:
trainer.push_to_hub()

Saving model checkpoint to clinico
Configuration saved in clinico/config.json
Model weights saved in clinico/pytorch_model.bin
tokenizer config file saved in clinico/tokenizer_config.json
Special tokens file saved in clinico/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/255M [00:00<?, ?B/s]

Upload file runs/Mar11_20-10-09_minion/events.out.tfevents.1678561872.minion.3412979.0:  22%|##2       | 32.0k…

To https://huggingface.co/joheras/clinico
   ab2f7bc..e85b72c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.3056282722513089}, {'name': 'Recall', 'type': 'recall', 'value': 0.5188888888888888}, {'name': 'F1', 'type': 'f1', 'value': 0.38467874794069185}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8225033664234491}]}
To https://huggingface.co/joheras/clinico
   e85b72c..cb7cc23  main -> main



'https://huggingface.co/joheras/clinico/commit/e85b72c85bc3eae677829d250d4736241adbc78f'